In [ ]:
import numpy as np
import os
import sys
import json

sys.path.append('./figures/MAM2EBRAINS')

from multiarea_model import MultiAreaModel
from multiarea_model import Analysis
from M2E_compute_pop_rates import compute_pop_rates

base_path = os.path.abspath(".")
data_path = os.path.abspath("simulations")

In [ ]:
import sys
print("Python version:", sys.version)

In [ ]:
# Set the threshold of matching rate of mean firing rate of all populations
# if smaller than this value, raise error so that the test won't pass and something is wrong and needs to be checked manually
match_rate_threshold = 0.95

In [ ]:
"""
Down-scaled model.
Neurons and indegrees are both scaled down to 10 %.
Can usually be simulated on a local machine.

Warning: This will not yield reasonable dynamical results from the
network and is only meant to demonstrate the simulation workflow.
"""
d = {}
conn_params = {'replace_non_simulated_areas': 'het_poisson_stat',
               'cc_weights_factor': 1.9, # run model in Ground State
               'cc_weights_I_factor': 2.0}
network_params = {'N_scaling': 0.006,
                  'K_scaling': 0.006,
                  'fullscale_rates': os.path.join(base_path, 'tests/fullscale_rates.json')}

sim_params = {'t_sim': 2000.,
              'num_processes': 1,
              'local_num_threads': 1}

M = MultiAreaModel(network_params, simulation=True,
                   sim_spec=sim_params,
                   theory=True)

M.simulation.simulate()

In [ ]:
# Create an instance of Analysis to load data
A = Analysis(M, M.simulation, data_list=['spikes'], load_areas=None)

In [ ]:
# Create pop_rates, load stationary firing rates, and calculate mean firing rate for all populations
label = M.simulation.label
complete_area_list = ['V1', 'V2', 'VP', 'V3', 'V3A', 'MT', 'V4t', 'V4', 'VOT', 'MSTd',
                      'PIP', 'PO', 'DP', 'MIP', 'MDP', 'VIP', 'LIP', 'PITv', 'PITd',
                      'MSTl', 'CITv', 'CITd', 'FEF', 'TF', 'AITv', 'FST', '7a', 'STPp',
                      'STPa', '46', 'AITd', 'TH']

area_list = complete_area_list

compute_pop_rates(M, data_path, label) # Compute pop_rates

fn = os.path.join(data_path, label, 'Analysis', 'pop_rates.json') # Load stationary firing rates
with open(fn, 'r') as f:
    pop_rates = json.load(f)

# Process spike data and calculate mean firing rate for all populations, save them in a dict
rates = np.zeros((len(area_list), 8))
for i, area in enumerate(area_list):
    for j, pop in enumerate(M.structure[area][::-1]):
        rate = pop_rates[area][pop]
        if rate == 0.0:
            rate = 1e-5
        if area == 'TH' and j > 3:  # To account for missing layer 4 in TH
            rates[i][j + 2] = rate
        else:
            rates[i][j] = rate

rates = np.transpose(rates)
rates = np.array(rates)

In [ ]:
# Compare the calculated mean firing rate with the reference values saved in M2E_reference_MFRs.json and calculate the percentage of matching
# Load the array
pop_rates_ref = np.load('M2E_pop_rates_ref.npy')

# Count the percentage of cells that values match (the same)
comparison_result = pop_rates_ref == rates
# print(comparison_result)
same_cells = np.sum(comparison_result)
# print(same_cells)
total_cells = np.prod(pop_rates_ref.shape)
# print(total_cells)
match_rate = same_cells / total_cells
# print(match_rate)

# Judge if the matching rate is lower than threshold set and raise error if yes
if match_rate < match_rate_threshold:
    raise TestError("Mismatch of mean firing rates over populations between the latest simulation and saved reference values, there may be problems of recent updates of dependencies, please take a check!")